# Curve Statistics and Characterization

In [5]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy import signal

# load curves 
paths = glob.glob("/Users/juliebuchnielsen/Desktop/01666/AlignedCurves/Curve*_aligned.csv")
AllCurves = [np.loadtxt(path, delimiter=",") for path in paths]

In [6]:
%matplotlib qt

# plot all curves
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(projection="3d")

for i, curve in enumerate(AllCurves):
    ax.plot(*curve.T, label=f"Curve{i+1}")

ax.set_title("Alle alignede kurver (hver sin farve)", fontsize=14)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize='small')
plt.show()

## Characterization

In [8]:
# curve length

def curve_length(curve):
    """
    Compute the length of a 3D curve.
    
    Input:
    - curve [N x 3]: numpy array with 3d curve

    Output:
    - length (float): length of the given curve

    """
    # compute the displacement between each point:
    dis = np.diff(curve, axis=0)

    # compute sum of euclidian norms
    length = np.sum(np.linalg.norm(dis, axis=1))
    return length

def tortuosity(curve):
    """
    Compute the Tortuosity of a curve
    """
    euclid_dist = np.linalg.norm(curve[-1, :] - curve[0, :])
    T = curve_length(curve) / euclid_dist
    return T

Lengths = []
Tortuosity = []

for curve in AllCurves:
    # curve length
    Lengths.append(curve_length(curve))

    # curve tortuosity
    Tortuosity.append(tortuosity(curve))


# plot histogram

# plot length/Tortuosity

In [ ]:
# Signal analysis

# den her funktion er super sej, men ikke særlig stabil
def signal_oscillations(curve):
    """
    Compute the oscillations along the main axis of the data. 
    """
    centered = curve - curve.mean(axis=0)
    U, S, Vt = np.linalg.svd(centered)
    main_axis = Vt[0]
    
    t = centered @ main_axis
    projection = np.outer(t, main_axis)
    d = np.linalg.norm(centered - projection, axis=1) # orthogonal distance to main axis.

    peaks, _ = signal.find_peaks(d)
    troughs, _ = signal.find_peaks(-d)
    return len(peaks) + len(troughs), peaks, troughs, main_axis, d

curve_no = 1
curve = AllCurves[curve_no]
origin = curve.mean(axis=0)
axis_length = np.linalg.norm(curve.ptp(axis=0)) * 0.5
osc, peaks, troughs, axis, deviations = signal_oscillations(curve)

# amplitude
amp = np.max(deviations)

# plots
print(f"No. oscillations: {osc}")

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection="3d")

ax.plot(*curve.T, color='black', label=f'Curve{curve_no}')
ax.scatter(*curve[peaks].T, color='red', label='Peaks', s=50)
ax.scatter(*curve[troughs].T, color='blue', label='Throughs', s=50)
ax.quiver(*origin, *(axis * axis_length), color='green', linewidth=2, arrow_length_ratio=0.1, label='Hovedakse')

ax.set_title("Kurve med identificerede peaks og throughs")
ax.legend()
plt.tight_layout()
plt.show()

No. oscillations: 17


In [23]:
plt.figure(figsize=(6, 3))
plt.plot(deviations, color='black')
plt.scatter(peaks, deviations[peaks], color='red')
plt.scatter(troughs, deviations[troughs], color='blue')
plt.title("Deviation fra hovedakse")
plt.xlabel("Punkt langs kurven")
plt.ylabel("Afstand fra akse")
plt.tight_layout()
plt.show()


In [ ]:
# frequency

In [ ]:
# variance between curves.